# Step 1: download azure cli

https://learn.microsoft.com/en-us/cli/azure/install-azure-cli

# Step 2: Log in with az login 

https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli

# Step 3: Create python virtual env we have 3.10
Packages:

azure-common==1.1.28

azure-core==1.26.1

azure-identity==1.12.0

azure-keyvault==4.2.0

azure-keyvault-secrets==4.6.0

azure-storage-blob==12.14.1

In [1]:
from azure.keyvault.secrets import SecretClient
from azure.storage.blob import BlobClient, BlobServiceClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
import sys


keyvault_url = 'https://<value from virginia>.vault.azure.net/'
secret_name = '<value from virginia>'

def get_az_credential():
    """Obtains an Azure credential, based on the environment you run from (Linux, Windows (local vs DSVM))
    
    Parameters:
    None
    
    Returns:
    - Credential: Credential for connecting to Azure
    """
    credential = AzureCliCredential()
    
    return credential

# Connect to keyvault and fetch secret
def get_keyvault_secret(keyvault_url,secret_name, credential):
    """Get a secret from the keyvault
    
    Parameters:
    - keyvault_url (string): The URL of the storage account.
    - secret_name (string): The name of the container in the storage account.
    - credential (object): The credentials to be used for authentication.
    
    Returns:
    - String containing the secret
    """
    client = SecretClient(vault_url=keyvault_url, credential=credential)
    return client.get_secret(secret_name)

In [7]:
credential = get_az_credential()
storage_account_key = get_keyvault_secret(keyvault_url, secret_name , credential)

In [ ]:
from azure.storage.blob import BlobServiceClient
def list_blobs_in_container(storage_account_name, storage_account_key, container_name):
    try:
        # Construct the connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key.value};EndpointSuffix=core.windows.net"
        
        # Create the BlobServiceClient object
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)

        # List blobs in the container
        blob_list = container_client.list_blobs()
        
        print(f"Blobs in container '{container_name}':")
        for blob in blob_list:
            print(f"\t{blob.name}")

    except Exception as ex:
        print(f"Exception: {ex}")

# Example usage
storage_account_name = "<value from slack>"
container_name = "<value from slack>"

list_blobs_in_container(storage_account_name, storage_account_key, container_name)

In [38]:
import os

def download_blobs_in_container(storage_account_name, storage_account_key, container_name):
    try:
        # Construct the connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key.value};EndpointSuffix=core.windows.net"

        # Create the BlobServiceClient object
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)

        # List blobs in the container
        blob_list = container_client.list_blobs()

        for blob in blob_list:
            if not blob.name.endswith(".jpg"):
                continue
            name = blob.name.replace("/", "\\")
            local_path = os.path.join(r'<path to store>', name)

            # Ensure the directory exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            with open(local_path, mode="wb") as sample_blob:
                download_stream = container_client.download_blob(blob.name)
                sample_blob.write(download_stream.readall())

            print(f"Downloaded {blob.name} to {local_path}")


    except Exception as ex:
        print(f"Exception: {ex}")


In [ ]:
storage_account_name = "<value from slack>"
container_name = "<value from slack>"
download_blobs_in_container(storage_account_name, storage_account_key, container_name)